In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import altair as alt

In [6]:
perf = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data')
perf.head()

,adviser,32/60,125,256,6000,256.1,16,128,198,199
0,amdahl,470v/7,29,8000,32000,32,8,32,269,253
1,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
2,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
3,amdahl,470v/7c,29,8000,16000,32,8,16,132,132
4,amdahl,470v/b,26,8000,32000,64,8,32,318,290


In [7]:
perf.columns = ['vendor name','Model Name','MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX','PRP','ERP']

In [8]:
#perf.describe()

In [9]:
le = LabelEncoder()
le1 = LabelEncoder()

In [23]:
perf[perf['PRP'].gt(150)].head()

,vendor name,Model Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP,Vendor Name,model name
0,amdahl,470v/7,29,8000,32000,32,8,32,269,253,0,61
1,amdahl,470v/7a,29,8000,32000,32,8,32,220,253,0,62
2,amdahl,470v/7b,29,8000,32000,32,8,32,172,253,0,63
4,amdahl,470v/b,26,8000,32000,64,8,32,318,290,0,65
5,amdahl,580-5840,23,16000,32000,64,16,32,367,381,0,73


In [11]:
le.fit(perf['vendor name'])
le1.fit(perf['Model Name'])
Label = le.transform(perf['vendor name'])
Label1 = le1.transform(perf['Model Name'])
perf['Vendor Name'] = Label
perf['model name'] = Label1
perf1 = perf.drop(['vendor name'],axis =1)
perf2 = perf1.drop(['Model Name'],axis = 1)
#perf3 = perf2.drop(perf2.index[[8,197,198]])
perf2.head()

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP,Vendor Name,model name
0,29,8000,32000,32,8,32,269,253,0,61
1,29,8000,32000,32,8,32,220,253,0,62
2,29,8000,32000,32,8,32,172,253,0,63
3,29,8000,16000,32,8,16,132,132,0,64
4,26,8000,32000,64,8,32,318,290,0,65


In [12]:
X = perf2[['Vendor Name','MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX','model name']]
y = perf2['PRP']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.3,random_state = 0)

In [13]:
xgb = XGBRegressor(n_estimators =150,gamma =0.1,max_depth = 5)

In [14]:
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
pred = xgb.predict(X_test)

In [16]:
print(xgb.score(X_train,y_train)*100)
print(xgb.score(X_test,y_test)*100)

99.9746322345
94.1387055366


In [17]:
y_test[:5]

12     138
80      38
33     106
5      367
187    113
Name: PRP, dtype: int64

In [18]:
prediction = pred[:5]
prediction

array([  98.53533936,   29.45497131,  206.66656494,  486.60220337,
         96.22688293], dtype=float32)

In [19]:
corr1 = perf2['MMAX'].corr(perf2['PRP'])
corr1

0.86557615103818475

In [20]:
corr2 = perf2['CHMIN'].corr(perf2['PRP'])
corr2

0.60884107192115533

In [21]:
#alt.Chart(perf2).mark_circle().encode( alt.X(''),alt.Y('PRP'))

In [22]:
for i in range(6):
    corre = perf2.iloc[:,i].corr(perf2['PRP'])
    i=+1
    print(corre)   
    

-0.306571367979
0.798310467657
0.865576151038
0.704642441639
0.608841071921
0.621309107389
